In [1]:
import networkExpansionPy.lib as ne
from scipy.sparse import csr_matrix
import numpy as np
from random import sample
import pandas as pd

In [2]:
metabolism = ne.GlobalMetabolicNetwork()
#metabolism.pruneUnbalancedReactions()
metabolism.pruneInconsistentReactions()
#metabolism.set_ph(7.0)
metabolism.convertToIrreversible()
#metabolism.setMetaboliteBounds(ub=1e-1,lb=1e-6)
#metabolism.pruneThermodynamicallyInfeasibleReactions(keepnan=False)

# construct params for network expansion
network = metabolism.network.pivot_table(index='cid',columns = ['rn','direction'],values='s').fillna(0)
S = network.values
R = (S < 0)*1
P = (S > 0)*1
b = sum(R)

# sparsefy data
R = csr_matrix(R)
P = csr_matrix(P)
b = csr_matrix(b)
b = b.transpose()


In [18]:
rn_index = pd.DataFrame(list(network),columns = ['rn','direction']).groupby('rn').min()[[]]

In [3]:
fixed_metabolites = ['C00001','C00011','C00080','C00014','C00009','C00283']
seedSet = fixed_metabolites
x0 = np.array([x in seedSet for x in network.index.get_level_values(0)]) * 1
x0 = csr_matrix(x0)
x0 = x0.transpose()
met_trace,reaction_trace = ne.netExp_trace(R,P,x0,b)
rxn_iter = ne.parse_reaction_trace(reaction_trace,network)

/Users/joshuagoldford/miniconda2/envs/network_expansion/lib/python3.6/site-packages/scipy/sparse/compressed.py:226: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  " != instead.", SparseEfficiencyWarning)


In [9]:
rxn_iter.to_csv('6seedReactionIter.Nov2020.csv')

In [27]:

size_of_random_sample = 6
fixed_metabolites = ['C00001','C00011','C00080','C00014','C00009','C00283']
mets_population = [x for x in network.index.get_level_values(0).tolist() if x not in fixed_metabolites]

numSims = 10000;
sims = []

df = rn_index.copy()

# define iterations to save dataframe:
saveIter = np.linspace(0,numSims,101)

for i in range(numSims):
    
    seedSet = sample(mets_population,size_of_random_sample) + fixed_metabolites
    x0 = np.array([x in seedSet for x in network.index.get_level_values(0)]) * 1
    x0 = csr_matrix(x0)
    x0 = x0.transpose()
    met_trace,reaction_trace = ne.netExp_trace(R,P,x0,b)
    rxn_iter = ne.parse_reaction_trace(reaction_trace,network)
    sim = rxn_iter.groupby('rn').min()['iter']
    sim = pd.DataFrame(sim)
    sim.columns = [i]
    df = df.join(sim)
    #save simulations
    if i in saveIter:
        df.to_hdf('ne_results.12seedSet.ConsistentNetwork.11222020.temp.hdf','df')
        print('finished with ' + str(i)  + ' iteration and saved df')
    

df.to_hdf('ne_results.12seedSet.ConsistentNetwork.11222020.hdf','df')
#sims = pd.concat(sims,axis=1)
#sims.to_csv('randNetworkExpansion.Full.CHOSNPfixedMets.1e4samples.6mets.11202020.csv')

/Users/joshuagoldford/miniconda2/envs/network_expansion/lib/python3.6/site-packages/scipy/sparse/compressed.py:226: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  " != instead.", SparseEfficiencyWarning)


finished with 0 iteration and saved df
finished with 100 iteration and saved df
finished with 200 iteration and saved df
finished with 300 iteration and saved df
finished with 400 iteration and saved df
finished with 500 iteration and saved df
finished with 600 iteration and saved df
finished with 700 iteration and saved df
finished with 800 iteration and saved df
finished with 900 iteration and saved df
finished with 1000 iteration and saved df
finished with 1100 iteration and saved df
finished with 1200 iteration and saved df
finished with 1300 iteration and saved df
finished with 1400 iteration and saved df
finished with 1500 iteration and saved df
finished with 1600 iteration and saved df
finished with 1700 iteration and saved df
finished with 1800 iteration and saved df
finished with 1900 iteration and saved df
finished with 2000 iteration and saved df
finished with 2100 iteration and saved df
finished with 2200 iteration and saved df
finished with 2300 iteration and saved df
fini

In [29]:
#df
df.to_hdf('ne_results.12seedSet.ConsistentNetwork.11222020.final.hdf','df')

In [31]:
df = df.T

In [33]:
rxns = list(df)

In [37]:
df.max().max()

79.0

In [48]:
cycleNumToCount = {'rn':[],'cycle':[],'count':[]}
for i in range(1,80):
    for rxn in rxns:
        numOccur = sum(df[rxn] == i)
        cycleNumToCount['rn'].append(rxn)
        cycleNumToCount['cycle'].append(i)
        cycleNumToCount['count'].append(numOccur)

In [49]:
cycleNumToCount = pd.DataFrame(cycleNumToCount)

In [51]:
cycleNumToCount.to_csv('ne_results.12seedSet.ConsistentNetwork.11222020.cycleCount.csv')

In [54]:
cn = cycleNumToCount.pivot_table(index='cycle',columns='rn',values='count')

In [57]:
p = cn / 10000

In [64]:
import seaborn as sns

1.0